# Анализ сервиса книг

Куплен крупный сервис для чтения книг по подписке.

# Цели:
- Провести анализ сервиса
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Загрузка данных

In [1]:
!pip install psycopg2-binary

In [2]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': '# имя пользователя', # имя пользователя
'pwd': '# пароль', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': '# название базы данных'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Предобработка данных

- Формруем запросы и выводим данные:

In [4]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
         return pd.read_sql(sql=sa.text(query), con = con)

In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Cколько книг вышло после 1 января 2000 года?

In [9]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


После 01.2000 вышло 819 книг.

## Количество обзоров и средняя оценка каждой книги

In [10]:
query = '''
SELECT b.book_id, title, review_cnt, avg_rating
    FROM books AS b
    LEFT JOIN (SELECT book_id, COUNT(review_id) AS review_cnt FROM reviews GROUP BY book_id) AS re ON b.book_id=re.book_id
    JOIN (SELECT book_id, ROUND(AVG(rating), 2) AS avg_rating FROM ratings GROUP BY book_id)
    AS r ON b.book_id=r.book_id
    ORDER BY avg_rating DESC
    '''
get_sql_data(query)

,book_id,title,review_cnt,avg_rating
0,518,Pop Goes the Weasel (Alex Cross #5),2.0,5.00
1,732,The Ghost Map: The Story of London's Most Terr...,2.0,5.00
2,347,In the Hand of the Goddess (Song of the Liones...,2.0,5.00
3,610,Tai-Pan (Asian Saga #2),2.0,5.00
4,330,How to Be a Domestic Goddess: Baking and the A...,1.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,371,Junky,2.0,2.00
997,316,His Excellency: George Washington,2.0,2.00
998,202,Drowning Ruth,3.0,2.00


Добавлены столбцы средняя оценка и количество обзоров.

Самая высокая оценка Pop Goes the Weasel (Alex Cross #5) - 5

Самая низкая оценка Harvesting the Heart - 1,5

## Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [11]:
query = '''
SELECT p.publisher as publisher_name,
       COUNT(DISTINCT books.book_id) as books_count
FROM publishers as p
 LEFT JOIN books on p.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher_name
ORDER BY books_count DESC
LIMIT 10;
'''
get_sql_data(query)

,publisher_name,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,Delta,13


Издательство Penguin Books выпустило больше всех книг толще 50 страниц.

## Автор с самой высокой средней оценкой книг толще 50 страниц.

In [12]:
query = '''
SELECT authors.author,
       ROUND(AVG(ratings.rating),2) AS avg_rating
FROM ratings
 LEFT JOIN books AS books ON ratings.book_id=books.book_id
 LEFT JOIN authors AS authors ON books.author_id=authors.author_id
WHERE ratings.book_id IN (SELECT ratings.book_id
                          FROM ratings
                          GROUP BY ratings.book_id
                          HAVING COUNT(ratings.book_id) >= 50)
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1;
'''
get_sql_data(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг толще 50 страниц - J.K. Rowling/Mary GrandPré	

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [13]:
query = '''
SELECT AVG(rv.review_count) AS avg
FROM
    (SELECT username,
            COUNT(review_id) AS review_count
     FROM reviews
     WHERE username IN (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating_id) > 48
                       )
     GROUP BY username
    ) AS rv;
'''
get_sql_data(query)

,avg
0,24.0


Среднее количество обзоров от пользователей поставивши больше 48 оценок - 24.

# Выводы:
 - После 01.2000 вышло 819 книг.
 - Самая высокая оценка Pop Goes the Weasel (Alex Cross #5) - 5
 - Самая низкая оценка Harvesting the Heart - 1,5
 - Издательство Penguin Books выпустило больше всех книг толще 50 страниц.
 - Автор с самой высокой средней оценкой (4.29) книг толще 50 страниц - J.K. Rowling/Mary GrandPré
 - Среднее количество обзоров от пользователей поставивши больше 48 оценок - 24.